<h1> Random Forest Classifier

In [4]:
import pandas as pd
import numpy as np
from random import randrange

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
pd.set_option('display.max_columns', 500)

In [6]:
filename = 'cleaned_fraudTrain.csv'

In [8]:
# import Dataset
path = '../dataset/midterm_dataset/'+filename
df = pd.read_csv(path)

row_num = df.shape[0]
feature_num = df.shape[1]

print(df.shape)
df

(1296675, 14)


,category,amt,is_male,lat,long,city_pop,job,unix_time,is_fraud,state_city,age,trans_time,trans_year,trans_month
0,misc_net,4.97,0,36.0788,-81.1781,3495,"Psychologist, counselling",1325376018,0,"28654, Moravian Falls, NC",31,Late Night,2019,1
1,grocery_pos,107.23,0,48.8878,-118.2105,149,Special educational needs teacher,1325376044,0,"99160, Orient, WA",41,Late Night,2019,1
2,entertainment,220.11,1,42.1808,-112.2620,4154,Nature conservation officer,1325376051,0,"83252, Malad City, ID",57,Late Night,2019,1
3,gas_transport,45.00,1,46.2306,-112.1138,1939,Patent attorney,1325376076,0,"59632, Boulder, MT",52,Late Night,2019,1
4,misc_pos,41.96,1,38.4207,-79.4629,99,Dance movement psychotherapist,1325376186,0,"24433, Doe Hill, VA",33,Late Night,2019,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1296670,entertainment,15.56,1,37.7175,-112.4777,258,Geoscientist,1371816728,0,"84735, Hatch, UT",59,Morning,2020,6
1296671,food_dining,51.70,1,39.2667,-77.5101,100,"Production assistant, television",1371816739,0,"21790, Tuscarora, MD",41,Morning,2020,6
1296672,food_dining,105.93,1,32.9396,-105.8189,899,Naval architect,1371816752,0,"88325, High Rolls Mountain Park, NM",53,Morning,2020,6
1296673,food_dining,74.90,1,43.3526,-102.5411,1126,Volunteer coordinator,1371816816,0,"57756, Manderson, SD",40,Morning,2020,6


<h2> Normalization

In [10]:
nor_col = ['amt', 'city_pop','lat', 'long', 'unix_time', 'age']
        
df[nor_col] = df[nor_col].apply(lambda x: (x - x.min()) / (x.max() - x.min()))
df

,category,amt,is_male,lat,long,city_pop,job,unix_time,is_fraud,state_city,age,trans_time,trans_year,trans_month
0,misc_net,0.000137,0,0.343968,0.864638,0.001194,"Psychologist, counselling",0.000000e+00,0,"28654, Moravian Falls, NC",0.207317,Late Night,2019,1
1,grocery_pos,0.003670,0,0.618450,0.485682,0.000043,Special educational needs teacher,5.598526e-07,0,"99160, Orient, WA",0.329268,Late Night,2019,1
2,entertainment,0.007569,1,0.474727,0.546553,0.001421,Nature conservation officer,7.105821e-07,0,"83252, Malad City, ID",0.524390,Late Night,2019,1
3,gas_transport,0.001520,1,0.561509,0.548070,0.000659,Patent attorney,1.248902e-06,0,"59632, Boulder, MT",0.463415,Late Night,2019,1
4,misc_pos,0.001415,1,0.394153,0.882190,0.000026,Dance movement psychotherapist,3.617509e-06,0,"24433, Doe Hill, VA",0.231707,Late Night,2019,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1296670,entertainment,0.000503,1,0.379084,0.544346,0.000081,Geoscientist,9.999981e-01,0,"84735, Hatch, UT",0.548780,Morning,2020,6
1296671,food_dining,0.001751,1,0.412281,0.902174,0.000026,"Production assistant, television",9.999983e-01,0,"21790, Tuscarora, MD",0.329268,Morning,2020,6
1296672,food_dining,0.003625,1,0.276699,0.612486,0.000301,Naval architect,9.999986e-01,0,"88325, High Rolls Mountain Park, NM",0.475610,Morning,2020,6
1296673,food_dining,0.002553,1,0.499837,0.646029,0.000379,Volunteer coordinator,1.000000e+00,0,"57756, Manderson, SD",0.317073,Morning,2020,6


In [12]:
label = 'is_fraud'
label

'is_fraud'

In [14]:
features = df.loc[:, df.columns != label].columns.tolist()
print(features)

['category', 'amt', 'is_male', 'lat', 'long', 'city_pop', 'job', 'unix_time', 'state_city', 'age', 'trans_time', 'trans_year', 'trans_month']


<h2>Information Gain table from the train dataset

In [16]:
df_ig = pd.read_csv('info_gain_all_feature.csv')
df_ig = df_ig.rename(columns={'Unnamed: 0' : 'features'})
df_ig.set_index("features", inplace = True)
df_ig

,information_gain
features,
amt,0.051352
lat,0.051352
long,0.051352
city_pop,0.051352
unix_time,0.051352
age,0.051352
state_city,0.007758
trans_time,0.005617
job,0.003134


<h2>All necessary function

In [18]:
def find_prob_each(label, feature, df=df):
    """
    -helper function for group_entropy function
    -and also give probability often its occures compare to the label
    
    return dict of probability of feature
    """
    if feature not in df.columns.tolist():
        return {'never_met': [0, 0]}

    temp_df = df.groupby([feature, label])
    prob_list = []

    if len(temp_df.size())< 2000 and feature not in ['age', 'lat', 'long', 'city_pop']:  
        for r in range(len(temp_df.size())):
                feature_prob = temp_df.size().values[r]/ \
                    len(df[feature][df[feature] == temp_df.size().index[r][0]])
                prob_feature = {}
                prob_feature[temp_df.size().index[r]] = feature_prob
                featu = ''
                if len(prob_list) == 0:
                    featu = temp_df.size().index[r][0]
                    prob_list.append({featu : [0.0, 0.0]})
                    if temp_df.size().index[r][1] == 1:
                        prob_list[-1][featu][1] = feature_prob
                    else:
                        prob_list[-1][featu][0] = feature_prob
                elif temp_df.size().index[r][0] in prob_list[-1]: #append 2nd value
#                     print(df.groupby([feature, label]).size().index[r][0], r,feature_prob, temp_df.size().index[r][1])
#                     print(prob_list[-1][temp_df.size().index[r][0]])
                    if temp_df.size().index[r][1] == 1:
                        prob_list[-1][temp_df.size().index[r][0]][1] = feature_prob
                    else:
                        prob_list[-1][temp_df.size().index[r][0]][0] = feature_prob
                else:
                    featu = temp_df.size().index[r][0]
                    prob_list.append({featu : [0.0, 0.0]})
                    if temp_df.size().index[r][1] == 1:
                        prob_list[-1][featu][1] = feature_prob
                    else:
                        prob_list[-1][featu][0] = feature_prob
                    
    else: #numeric feature
        greater_than_eq_df = df[[label, feature]][df[feature] >=df[feature].mean()]
        less_than_df = df[[label, feature]][df[feature] < df[feature].mean()]
        condition_dfs = [greater_than_eq_df, less_than_df]
    
        prob_list = [{'greater_than_eq_mean': []}, {'less_than_mean': []}]
    
        for d in condition_dfs:
            for r in range(len(d[label].value_counts())):
                feature_prob = d[label].value_counts()[r]/len(d)
#                 print(feature_prob)
                if  d.equals(greater_than_eq_df):
                    prob_list[0]['greater_than_eq_mean'].append(feature_prob)

                else:
                    prob_list[1]['less_than_mean'].append(feature_prob)
    p_dict = {}
    [p_dict.update(i) for i in prob_list]
    
    return p_dict

In [20]:
#containing dict to decrease workload of model
store_prob_dict = {}

In [109]:
def extract_feature_point(feature, value, df=df, store_prob_dict=store_prob_dict):

    if feature not in df.columns.tolist():
        return [0,0]
    #check type
    if type(value) != str and feature not in ['is_male', 'trans_year', 'trans_month']:
        mean_val = df[feature].mean()
        if value >= mean_val:
            value = 'greater_than_eq_mean'
        else:
            value = 'less_than_mean'
    
    print(feature, 'value', value)
        
    if feature not in store_prob_dict:
        prob_dict = find_prob_each(label, feature, df)
        store_prob_dict.update({feature : prob_dict})
        print(feature)
        print('store_dict length:', len(store_prob_dict))
        
    
    #append value
    if value not in store_prob_dict[feature]:
        #return 0 when hv no data
        return [0, 0]
    elif value in store_prob_dict[feature]:
        # print(store_prob_dict[feature])
        #return point
        return [
            100*store_prob_dict[feature][value][0]*float(df_ig.loc[feature,:]),
            100*store_prob_dict[feature][value][1]*float(df_ig.loc[feature,:])
        ]

In [110]:
def row_desicion_tree_predictor(an_row=[], features=[], df=df, store_prob_dict=store_prob_dict, x_features=features):
    not_fraud = []
    fraud = []
#     an_row = an_df_row.values.tolist()[0]
    #making index of sub sample and test data to be the same
    #in case x_features lenght is more than df features
    for i, feature in enumerate(x_features):
        point = extract_feature_point(feature, an_row[i], df, store_prob_dict)
        not_fraud.append(point[0])
        fraud.append(point[1])
#         print(feature,':',point,'done')
    print('predicting..')
#     print(1)
    #predicting
    nfra_mean = np.mean(not_fraud)
    fra_mean = np.mean(fraud)
    if nfra_mean >= fra_mean:
        return 0
    else:
        return 1

In [106]:
def desicion_tree_predictor(X, train, features, store_prob_dict={}):
    """
    return list of predicting point
    """
    x_features = X.loc[:, X.columns != label].columns.tolist()
    x_list = X.values.tolist()
    return [row_desicion_tree_predictor(row, features, train, store_prob_dict, x_features) for row in x_list]

<h2> Random Forest preparation

In [24]:
subset_features_per_df_list = []
def subsample(features, ratio):
    selected_features = list()
    n_sample = round(len(features) * ratio)
    print(n_sample)
    while len(selected_features) < n_sample:
        index = randrange(len(features))
        selected_features.append(features[index])
    selected_features = list(set(selected_features))
    return selected_features

In [96]:
#random subset for creating Tree
n_subset = 3 #amount of tree
subset_features_per_df_list = []
store_prob_dict_list = []
for n in range(n_subset):
    store_prob_dict_list.append({})
    subset_features_per_df_list.append(subsample(features, 0.8))
    subset_features_per_df_list[n].append(label)
subset_features_per_df_list

10
10
10


[['trans_time', 'long', 'city_pop', 'age', 'amt', 'unix_time', 'is_fraud'],
 ['is_male', 'trans_time', 'category', 'age', 'trans_year', 'job', 'is_fraud'],
 ['trans_time',
  'age',
  'trans_month',
  'city_pop',
  'state_city',
  'amt',
  'unix_time',
  'is_fraud']]

In [105]:
#containing dict to decrease workload of model
store_prob_dict_list

[{'trans_time': {'Early Morning': [0.9987882656830931, 0.0012117343169069448],
   'Eve': [0.9988477987997268, 0.001152201200273171],
   'Late Night': [0.9879008177410784, 0.01209918225892156],
   'Morning': [0.9989780755494148, 0.0010219244505852603],
   'Night': [0.9804176957103282, 0.019582304289671826],
   'Noon': [0.998771602198105, 0.0012283978018950237]},
  'long': {'greater_than_eq_mean': [0.9939216773621463, 0.006078322637853727],
   'less_than_mean': [0.9945934441710067, 0.005406555828993304]},
  'city_pop': {'greater_than_eq_mean': [0.9936363261008692,
    0.0063636738991308655],
   'less_than_mean': [0.9943123656406319, 0.005687634359368186]},
  'age': {'greater_than_eq_mean': [0.9930538271431827, 0.006946172856817345],
   'less_than_mean': [0.9951208092047732, 0.004879190795226849]},
  'amt': {'greater_than_eq_mean': [0.9864942104874991, 0.013505789512500923],
   'less_than_mean': [0.9980922290206041, 0.0019077709793958416]},
  'unix_time': {'greater_than_eq_mean': [0.99432

In [98]:
df_list = ()
df_list = [df[f] for f in subset_features_per_df_list]
df_list[0].head()

,trans_time,long,city_pop,age,amt,unix_time,is_fraud
0,Late Night,0.864638,0.001194,0.207317,0.000137,0.000000e+00,0
1,Late Night,0.485682,0.000043,0.329268,0.003670,5.598526e-07,0
2,Late Night,0.546553,0.001421,0.524390,0.007569,7.105821e-07,0
3,Late Night,0.548070,0.000659,0.463415,0.001520,1.248902e-06,0
4,Late Night,0.882190,0.000026,0.231707,0.001415,3.617509e-06,0


<h2> Import test data

In [99]:
# import Dataset
path = '../dataset/midterm_dataset/cleaned_fraudTest.csv'
df_test = pd.read_csv(path)

row_num = df_test.shape[0]
feature_num = df_test.shape[1]

print(df_test.shape)
df_test.head()

(555719, 14)


,category,amt,is_male,lat,long,city_pop,job,unix_time,is_fraud,state_city,age,trans_time,trans_year,trans_month
0,personal_care,2.86,1,33.9659,-80.9355,333497,Mechanical engineer,1371816865,0,"29209, Columbia, SC",52,Morning,2020,6
1,personal_care,29.84,0,40.3207,-110.4360,302,"Sales professional, IT",1371816873,0,"84002, Altonah, UT",30,Morning,2020,6
2,health_fitness,41.28,0,40.6729,-73.5365,34496,"Librarian, public",1371816893,0,"11710, Bellmore, NY",50,Morning,2020,6
3,misc_pos,60.05,1,28.5697,-80.8191,54767,Set designer,1371816915,0,"32780, Titusville, FL",33,Morning,2020,6
4,travel,3.19,1,44.2529,-85.0170,1126,Furniture designer,1371816917,0,"49632, Falmouth, MI",65,Morning,2020,6


<h2>Normalization test dataset

In [100]:
nor_col = ['amt', 'city_pop','lat', 'long', 'unix_time', 'age']
        
df_test[nor_col] = df_test[nor_col].apply(lambda x: (x - x.min()) / (x.max() - x.min()))
df_test

,category,amt,is_male,lat,long,city_pop,job,unix_time,is_fraud,state_city,age,trans_time,trans_year,trans_month
0,personal_care,0.000082,1,0.305255,0.867121,0.114727,Mechanical engineer,0.000000e+00,0,"29209, Columbia, SC",0.456790,Morning,2020,6
1,personal_care,0.001267,0,0.444423,0.565239,0.000096,"Sales professional, IT",4.785402e-07,0,"84002, Altonah, UT",0.185185,Morning,2020,6
2,health_fitness,0.001769,0,0.452136,0.942836,0.011860,"Librarian, public",1.674891e-06,0,"11710, Bellmore, NY",0.432099,Morning,2020,6
3,misc_pos,0.002594,1,0.187080,0.868312,0.018834,Set designer,2.990876e-06,0,"32780, Titusville, FL",0.222222,Morning,2020,6
4,travel,0.000096,1,0.530537,0.825355,0.000379,Furniture designer,3.110511e-06,0,"49632, Falmouth, MI",0.617284,Morning,2020,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
555714,health_fitness,0.001879,1,0.448199,0.755010,0.000171,Town planner,9.999984e-01,0,"63453, Luray, MO",0.481481,Night,2020,12
555715,kids_pets,0.004868,1,0.197364,0.718694,0.009879,Futures trader,9.999985e-01,0,"77566, Lake Jackson, TX",0.074074,Night,2020,12
555716,kids_pets,0.003772,0,0.573103,0.478609,0.001260,Musician,9.999989e-01,0,"99323, Burbank, WA",0.296296,Night,2020,12
555717,travel,0.000307,1,0.538697,0.503704,0.000036,Cartographer,9.999994e-01,0,"83643, Mesa, ID",0.493827,Night,2020,12


In [101]:
X_test = df_test[features]
X_test.head()

,category,amt,is_male,lat,long,city_pop,job,unix_time,state_city,age,trans_time,trans_year,trans_month
0,personal_care,0.000082,1,0.305255,0.867121,0.114727,Mechanical engineer,0.000000e+00,"29209, Columbia, SC",0.456790,Morning,2020,6
1,personal_care,0.001267,0,0.444423,0.565239,0.000096,"Sales professional, IT",4.785402e-07,"84002, Altonah, UT",0.185185,Morning,2020,6
2,health_fitness,0.001769,0,0.452136,0.942836,0.011860,"Librarian, public",1.674891e-06,"11710, Bellmore, NY",0.432099,Morning,2020,6
3,misc_pos,0.002594,1,0.187080,0.868312,0.018834,Set designer,2.990876e-06,"32780, Titusville, FL",0.222222,Morning,2020,6
4,travel,0.000096,1,0.530537,0.825355,0.000379,Furniture designer,3.110511e-06,"49632, Falmouth, MI",0.617284,Morning,2020,6


In [102]:
y_test = df_test[label]
y_test.head()

0    0
1    0
2    0
3    0
4    0
Name: is_fraud, dtype: int64

<h2>Predicting each tree

In [103]:
ypred_of_tree = []

In [108]:
for tree in range(n_subset):
    print('=================================================== Current Tree:', tree,'==================================================================')
    ypred_of_tree.append(
        desicion_tree_predictor(
            X=X_test,
            train=df_list[tree],
            features=df_list[tree].loc[:, df_list[tree].columns != label].columns.tolist(),
            store_prob_dict=store_prob_dict_list[tree]
        )
    )
    print('===================================================== Done Tree:', tree,'==================================================================')
ypred_of_tree

=================================================== Current Tree: 0 ==================================================================
category value personal_care
category
store_dict length: 7
amt value less_than_mean
is_male value 1
is_male
store_dict length: 8


KeyError: 'lat'